# DeepStar Convolutional Neural Network

In [1]:
import os, sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import time, onnx, loader, torch, torchvision
import torch.utils.data as utils
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter

from Model import DeepStar, PathDataLoader

Conv1: 3, 64
Conv2: 64, 32

Dropout might be bad idea, longer train might be needed to test

Test 1: Fail, platoued at 0.07 loss
Added droput layer
Conv1 output layers from 64 to 20
Conv2 input layesr from 64 to 20
Conv2 output layers from 32 to 10
Fc1 outputs from 64 to 20
Fc2 inputs from 64 to 20

Test 2: Fail, platoued at 0.08 loss
Added droput layer
Conv1 output layers from 64 to 128
Conv2 input layesr from 64 to 128
Conv2 output layers from 32 to 64

In [2]:
load = False
save = True
save_name = "deep_star"

image_size = (256, 256)

data_folder = "data"
batch_size = 10
n_epochs = 20000
shuffle_data = True

rho = 0.9
lr = 1
weight_decay = 0

seed = 0

In [3]:
net = DeepStar()

if (load):
    net.load_state_dict(torch.load(save_name))

## Load data

Here we load both the training data and validation data into different loaders. There are methods for automatically separating out training and validation data from a set but we do it manually.

In [4]:
train_data = PathDataLoader(f'{data_folder}/images/')
train_loader = utils.DataLoader(train_data, batch_size=batch_size, shuffle=shuffle_data, num_workers=4)

val_data = PathDataLoader(f'{data_folder}/validation/')
val_loader = utils.DataLoader(train_data, batch_size=batch_size, shuffle=shuffle_data, num_workers=4)

## Configure output for Tensor Board

Here we write to special files that enable Tensor Board to visualize it. To open Tensor Board select the data/images/ folder and click the Tensorboard button.

In [5]:
#writer = SummaryWriter(f'{data_folder}/images')

#dataiter = iter(train_loader)
#images, labels = dataiter.next()

#img_grid = torchvision.utils.make_grid(images)
#writer.add_image('dataset', img_grid)

#writer.add_graph(net, images)
#writer.close()

## Train The Model

In [6]:
n_batches = len(train_loader)

loss = net.get_loss()
optimizer = net.get_optim(rho, lr, weight_decay)
training_start_time = time.time()

print(f'Training with {n_batches} batches')
for epoch in range(n_epochs):

    running_loss = 0.0
    print_every = n_batches // 10
    start_time = time.time()
    total_train_loss = 0

    for i, data in enumerate(train_loader, 0):
        i, e = data
        
        optimizer.zero_grad()

        output = net(i)
        loss_size = loss(output, mid)
        loss_size.backward()
        optimizer.step()
        
        running_loss += loss_size.data.item()
        total_train_loss += loss_size.data.item()
        
        if (i + 1) % (print_every + 1) == 0:
            print("Epoch {}, {:d}% \t train_loss: {:.3f} took: {:.2f}s".format(epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))

            running_loss = 0.0
            start_time = time.time()
            
    total_val_loss = 0
    for inputs, labels in val_loader:
        img, expected = Variable(img), Variable(expected)

        val_outputs = net(img)
        val_loss_size = loss(val_outputs, expected)
        total_val_loss += val_loss_size.data.item()

    print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))

print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

Training with 752 batches


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<string>", line 19, in __getitem__
TypeError: 'float' object is not iterable


In [ ]:
model_input = torch.randn(20, 3, 256, 256)

if (save):
    torch.save(net.state_dict(), f'{save_name}')
    torch.onnx.export(net,
                model_input,
                f'{save_name}.onnx',
                  
                export_params=True,
                opset_version=10,
                do_constant_folding=True)
                  

In [ ]:
torch.manual_seed(seed)

point = (184, 170)
img = Image.open("data/{0}_{1}_confirm.png".format(point[0], point[1]))
to_tensor = transforms.ToTensor()
img_tensor = to_tensor(img).unsqueeze(0)

output = net(img_tensor)
print(output)
print(round(output[0][0].item() * 256), round(output[0][1].item() * 256))